# Dimensionality Reduction Excercise

In [1]:
import numpy as np
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=True)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist['data'].copy(), mnist['target'].copy()
display(X.shape)
display(y.shape)
display(X.info())

(70000, 784)

(70000,)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 784 entries, pixel1 to pixel784
dtypes: float64(784)
memory usage: 418.7 MB


None

In [4]:
y = y.astype(np.uint8)
display(y.info())

<class 'pandas.core.series.Series'>
RangeIndex: 70000 entries, 0 to 69999
Series name: class
Non-Null Count  Dtype
--------------  -----
70000 non-null  uint8
dtypes: uint8(1)
memory usage: 68.5 KB


None

In [5]:
X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

In [6]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=2022)

In [8]:
import time

t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()

In [9]:
print("Training took {:2f}s".format(t1-t0))

Training tok 42.385642s


In [10]:
from sklearn.metrics import accuracy_score

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.969

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

In [12]:
rnd_clf_pca = RandomForestClassifier(n_estimators=100, random_state=2022)

In [13]:
t0 = time.time()
rnd_clf_pca.fit(X_train_reduced, y_train)
t1 = time.time()
print("Training took {:2f}s".format(t1-t0))

Training tok 103.936339s


Training took longer than before PCA was applied.
It depends on dataset, the model and the training algorithm.

In [14]:
X_test_reduced = pca.transform(X_test)
y_pred = rnd_clf_pca.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9489

In [17]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=2022)
t0 = time.time()
log_clf.fit(X_train_reduced, y_train)
t1 = time.time()
print("Training took {:2f}s".format(t1-t0))

Training took 5.305878s


c:\Users\arrog\anaconda3\envs\Finance\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
